## Requirements

CUDNN from the [Nvidia website](https://developer.nvidia.com/cudnn)  
The dataset in the connected Google Drive

- Connection to the Google Drive containing the dataset
- Clone the darknet repo

In [0]:
ROOT = "/content/drive/My Drive/Private/MA3 Project/"

import os
from google.colab import drive

drive.mount('/content/drive/')
os.chdir( ROOT )

!git clone https://github.com/josueBulle/darknet.git

os.chdir( os.path.join( ROOT, "darknet") )

Installation of CUDA and CUDNN so we can compile darknet with the GPU=1 option.  
We require the file  cudnn-8.0-linux-x64-v7.tgz  which is downloadable from the [Nvidia website](https://developer.nvidia.com/cudnn) (requires a developper account)

I used cuDNN v7.0.5 (Dec 5, 2017), for CUDA 8.0

In [0]:
# can take up to 7 minutes

!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub
!apt update
!apt install cuda-8-0 -y
os.environ['PATH'] += ':/usr/local/cuda/bin'
!apt -o Dpkg::Options::="--force-overwrite" --fix-broken install
!apt install gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++
os.chdir("/content/drive/My Drive/Private/MA3 Project/")
!tar -xzvf cudnn-8.0-linux-x64-v7.tgz
!cp cuda/include/cudnn.h /usr/local/cuda/include
!cp cuda/lib64/libcudnn* /usr/local/cuda/lib64
!chmod a+r /usr/local/cuda/include/cudnn.h

In [0]:
os.chdir( os.path.join( ROOT, "darknet") )
!make

# Transformations needed on dataset
YOLOv3 need one txt file per image (currently one per video). Each row must represent one groundtruth boundingbox with format:


```
<object-class> <center-x> <center-y> <width> <height>
```
The class in an integer between 0 and #classes - 1
Other values are relative to the image size (i.e. between 0.0 and 1.0)

```
<class_number> (<absolute_x> / <image_width>) (<absolute_y> / <image_height>) (<absolute_width> / <image_width>) (<absolute_height> / <image_height>)
```


**It took me about 30 minutes to transform the whole dataset (~24270 images)**


In [0]:
%cd "/path/to/dataset"

from DataManagerBenchmark import DataManager
from IPython.display import HTML, display

def progress(value=0, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [0]:
out = display(progress(), display_id=True)

def callback(a, b):
    out.update(progress(a, b))

dataManager = DataManager("UAV-benchmark/images", "UAV-benchmark/gt")
dataManager.export_dataset("UAV-benchmark/yolo", False, 0, 1e5, callback)

#18h57

# Prepare training

Compute best anchors for our custom dataset

In [0]:
! ./darknet detector anchors cfg/aerial-tiny.data -num_of_clusters 9 -width 416 -height 416

# Finally : Training

In [0]:
! ./darknet detector train cfg/aerial.data cfg/yolov3-aerial.cfg ../backup/yolov3-aerial.backup

# Testing detection

In [0]:
! ./darknet detector test cfg/aerial.data cfg/yolov3-aerial.cfg ../backup/yolov3-aerial_500.weights
# data/test1.jpg

In [0]:
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
test_image = mpimg.imread(os.path.join("predictions.jpg"))
plt.imshow(test_image)
plt.show()

Detection on a video

In [0]:
from python.helpers import *
import python.darknet as dn
import os, cv2

## Export a directory of images to a single output video

input_dir = "../benchmark-dataset/M0603/"
output_dir = "../output/"

output_name = "benchmark-m0603"

cfg = "cfg/jekhor/yolov3-aerial.cfg"
weights = "../yolov3-aerial.weights"
data = "cfg/jekhor/aerial.data"

##

print(os.listdir( input_dir ))

output_path = os.path.join(output_dir, output_name + ".mp4")

if not os.path.isdir(output_dir + output_name):
  os.makedirs(output_dir + output_name)

images = [ d for d in os.listdir( input_dir ) if d.endswith('.jpg') ].sort()
out_images = []

net = dn.load_net( cfg.encode(), weights.encode(), 0)
meta = dn.load_meta( data.encode() )

count = 0
total = len(images)
for i in images:
    count += 1
    print(str(count/total) + "%", end="\r")
    
    im = input_dir + i
    r = dn.detect(net, meta, im.encode())
    
    im_out = draw_yolo_result(im, r)
    
    out_images.append( os.path.join(output_dir, output_name, i) )
    
    cv2.imwrite( out_images[-1], im_out )

directory_to_video( out_images, output_dir )

